In [ ]:
import vtk
import numpy as np

In [ ]:
import vtk

# Read the point cloud from a file
reader = vtk.vtkPLYReader()
reader.SetFileName('point_cloud.ply')
reader.Update()

# Perform Poisson surface reconstruction
poisson = vtk.vtkPoissonReconstruction()
poisson.SetDepth(10)  # Set the depth parameter (controls the level of detail)
poisson.SetInputConnection(reader.GetOutputPort())
poisson.Update()

# Get the reconstructed surface mesh
surface = vtk.vtkPolyData()
surface.ShallowCopy(poisson.GetOutput())

# Write the surface mesh to a file
writer = vtk.vtkPLYWriter()
writer.SetFileName('reconstructed_surface.ply')
writer.SetInputData(surface)
writer.Write()


In [ ]:
# X = np.load('MasksForPlot.npy')
X = np.load('MasksNPYFiles/masksFinalContoured.npy')
# Y = X[:,:,:,0]
# Y = X[0:5,:,:,0]
Y = X[0:50,:,:]          #Set frames here
pts = np.argwhere(Y*1)
print(X.shape, X.size, Y.shape, Y.size)
print(len(pts.tolist()))
print(pts.tolist()[0:5])

In [ ]:
# def gen_points_sample():
    
#     pt = list()
    
#     for i in range(3,8):
#         for j in range(3,8):
#             for k in range(3,8):
#                 pt.append([i, j, k])
#     return pt

In [ ]:
def gen_points_sample():
    
    pt = pts.tolist()
    npts = list()
    for point in pt:
#         point.reverse()
        npts.append([point[2], point[1], point[0]])
    return pt

In [ ]:
# points = vtk.vtkPoints()
# print(len(gen_points_sample()))
# for point in gen_points_sample():
#     points.InsertNextPoint(point)

In [ ]:
points = vtk.vtkPoints()
print(len(gen_points_sample()))
for point in gen_points_sample():
    points.InsertNextPoint(point)

In [ ]:
# # Create a list of custom points
# points = vtk.vtkPoints()
# points.InsertNextPoint(0, 0, 0)
# points.InsertNextPoint(0, 3, 0)
# points.InsertNextPoint(0, 3, 13)
# points.InsertNextPoint(0, 3, 6)
# points.InsertNextPoint(7, 0, 0)
# points.InsertNextPoint(7, 0, 13)
# points.InsertNextPoint(7, 3, 13)
# points.InsertNextPoint(7, 3, 0)

In [ ]:
# Create an unstructured grid from the points
grid = vtk.vtkUnstructuredGrid()
grid.SetPoints(points)

In [ ]:
# Use the GPU tessellator to create a Delaunay triangulation
# tessellator = vtkGPUUnstructuredGridTessellator()
# tessellator.SetInputData(grid)
# tessellator.Update()

In [ ]:
# Create a point source and set its parameters
point_source = vtk.vtkPointSource()
point_source.SetNumberOfPoints(points.GetNumberOfPoints())
point_source.SetCenter(0, 0, 0)
point_source.SetRadius(1.0)

In [ ]:
# Set the custom points as the output of the point source
point_source.Update()
point_source.GetOutput().GetPoints().DeepCopy(points)

In [ ]:
# Perform Poisson surface reconstruction
poisson = vtk.vtkPoissonReconstruction()
poisson.SetDepth(10)  # Set the depth parameter (controls the level of detail)
poisson.SetInputConnection(point_source.GetOutput())
poisson.Update()

In [ ]:
# Get the reconstructed surface mesh
surface = vtk.vtkPolyData()
surface.ShallowCopy(poisson.GetOutput())

In [ ]:
# Use a Delaunay3D filter to generate a mesh from the points
delaunay = vtk.vtkDelaunay3D()
delaunay.SetInputData(point_source.GetOutput())
delaunay.Update()

In [ ]:
# Convert to a geometric filter
geom_filter = vtk.vtkGeometryFilter()
geom_filter.SetInputConnection(delaunay.GetOutputPort())
geom_filter.Update()

In [ ]:
# Create a mapper and actor for the mesh
mapper = vtk.vtkPolyDataMapper()
# mapper.SetInputData(point_source.GetOutput())
mapper.SetInputData(geom_filter.GetOutput())
actor = vtk.vtkActor()
actor.SetMapper(mapper)

In [ ]:
# Create a renderer and add the actor to it
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)

In [ ]:
# Create a render window and set its size
render_window = vtk.vtkRenderWindow()
render_window.SetSize(600, 600)

In [ ]:
# Add the renderer to the render window
render_window.AddRenderer(renderer)

# Create an interactor and start the render loop
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)
interactor.Initialize()
interactor.Start()